In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
# Set default plotly template
import plotly.io as pio
pio.templates.default = "plotly_white"


import plotly.express as px

STATE = 'thermal'
IMG_WIDTH = 600
IMG_HEIGHT = 350

# Set default plotly image format and dimensions
pio.kaleido.scope.default_format = "pdf"
#pio.kaleido.scope.mathjax= None
pio.kaleido.scope.default_width = IMG_WIDTH
pio.kaleido.scope.default_height = IMG_HEIGHT

STYLES = {
    'plotly': (
        '#636EFA',
        '#EF553B',
        '#00CC96',
        '#A4036f',
        '#0B0033',
        '#E5D352'
    ),
    'pastel': (
        '#66C5CC',
        '#F6CF71',
        '#F89C9C',
        '#DCB0F2',
    ),
    'pygal': (
        '#F44336',  # 0
        '#3F51B5',  # 4
        '#009688',  # 8
        '#FFC107',  # 13
        '#FF5722',  # 15
        '#9C27B0',  # 2
        '#03A9F4',  # 6
        '#8BC34A',  # 10
        '#FF9800',  # 14
        '#E91E63',  # 1
        '#2196F3',  # 5
        '#4CAF50',  # 9
        '#FFEB3B',  # 12
        '#673AB7',  # 3
        '#00BCD4',  # 7
        '#CDDC39',  # 11b
        '#9E9E9E',  # 17
        '#607D8B',  # 18
    )
}
LINESTYLES = ['-', (0, (6, 6)), '--', '-.', (0, (3, 5, 1, 5, 1, 5)), ':',]

mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=STYLES['plotly'], linestyle=LINESTYLES)
mpl.rcParams['axes.facecolor'] = 'white'
mpl.rcParams['axes.grid'] = True
mpl.rcParams['grid.color'] = 'lightgrey'
mpl.rcParams['grid.linestyle'] = '--'
mpl.rcParams['grid.linewidth'] = 0.5
mpl.rcParams['lines.linewidth'] = 2.5
mpl.rcParams['xtick.color'] = 'lightgrey'
mpl.rcParams['ytick.color'] = 'lightgrey'
mpl.rcParams['xtick.labelcolor'] = 'black'
mpl.rcParams['ytick.labelcolor'] = 'black'
mpl.rcParams['axes.edgecolor'] = 'lightgrey'
mpl.rcParams['text.usetex'] = False
mpl.rcParams['font.family'] = 'Times New Roman'
mpl.rcParams['font.size'] = 20
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['savefig.format'] = 'eps'
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'

# Create saving folder if it does not exists
if not os.path.exists(f'../img/obs_evolution/{STATE}'):
    os.makedirs(f'../img/obs_evolution/{STATE}')
    

#====================================================================

def stable_temperature(a, p):
    a = float(a)
    p = float(p)
    ga = 2*a**2
    gb = (1-a**2) * (1+np.cos(p))
    return - 1 / np.log(ga / gb)

In [ ]:
logs = pd.read_csv('../objects/saved/logs.csv', index_col=0, sep=';')
df = pd.read_pickle(f'../objects/saved/observables.pkl')
df.head()
logs

## To Do
- Plot T1/T2 or T1-T2 and compare with Mutual Information

## Temperatures

In [ ]:
ids_grid = np.array([['32Th', '37Th']])


In [ ]:
for a, b in df_to_plot.iloc[::-1].groupby('Log ID'):
    print(a)

In [ ]:
ids_grid = np.array([[['38Th'], ['32Th']]])
df_to_plot = df.loc[df['Log ID'].isin(ids_grid.flatten())].iloc[::-1]
fig, axs = plt.subplots(nrows=len(ids_grid), ncols=len(ids_grid[0]), figsize=(16, 6), layout='tight')
for i, (indx, group) in enumerate(df_to_plot.groupby('Log ID')):
    if indx in ids_grid[0, 0]:
        ax = axs[0]
        ax.set_ylabel('Temperature ($\hbar\omega$)', fontsize=28)
    else:
        ax = axs[1]
        ax.set_xlim(-2, 30)
    timedelta = logs.loc[indx, 'timedelta']
    temp = stable_temperature(logs.loc[indx, 'alpha'], logs.loc[indx, 'phi'])
    label = f"T = {temp} | $\phi$ = {logs.loc[indx, 'phi']}"
    l1 = f'$T_{"{S1}"}$'
    l2 = f'$T_{"{S2}"}$'
    ax.plot(group['Time'], group['T1'], label=l1)
    ax.plot(group['Time'], group['T2'], label=l2)
    leg = ax.legend(ncol=2, fontsize=24, borderpad=1, title=f"$\\alpha = 0.25$\n\n$\phi={logs.loc[indx, 'phi']}$\n", title_fontsize=24)
    leg._legend_box.align = "left"
    # ax.set_xlim(0, 300)
    ax.set_xlabel('Time ($n\Delta t$)', fontsize=28)
    
plt.show()

In [ ]:
# Now one figure at a time
for plot_id in ['38Th', '32Th']:
    fig, ax = plt.subplots(figsize=(8, 8))
    timedelta = logs.loc[plot_id, 'timedelta']
    phi = logs.loc[plot_id, 'phi']
    alpha = logs.loc[plot_id, 'alpha']
    temp = stable_temperature(logs.loc[plot_id, 'alpha'], logs.loc[plot_id, 'phi'])
    label = f"T = {temp} | $\phi$ = {phi}"
    l1 = f'$T_{"{S1}"}$'
    l2 = f'$T_{"{S2}"}$'
    data = df.loc[df['Log ID'] == plot_id]
    ax.plot(data['Time'], data['T1'], label=l1)
    ax.plot(data['Time'], data['T2'], label=l2)
    leg = ax.legend(ncol=2, fontsize=24, borderpad=1, title=f"$\\alpha = {alpha}$\n\n$\phi={phi}$\n", title_fontsize=24)
    leg._legend_box.align = "left"
    if plot_id == '32Th':
        ax.set_xlim(-1.4, 30)
    ax.set_xlabel('Time ($n\Delta t$, a.u.)', fontsize=28)
    ax.set_ylabel('Temperature ($\hbar\omega$)', fontsize=28)
    
    # Save figure
    fig_label = "heating" if plot_id == '38Th' else "cooling"
    fig.savefig(f"../img/obs_evolution/{STATE}/temperature_evolution_{fig_label}_1.eps")

In [ ]:
fig.savefig(f"../img/obs_evolution/{STATE}/temperature_evolution_HeatingAndCooling.eps")
fig.savefig(f"../img/obs_evolution/{STATE}/temperature_evolution_HeatingAndCooling.png")

## Temperature Ratio

In [ ]:
ids_grid = np.array([[['35Th', '36Th','37Th','38Th'], ['31Th', '32Th', '33Th', '34Th']]])
df_to_plot = df.loc[df['Log ID'].isin(ids_grid.flatten())].iloc[::-1]
fig, axs = plt.subplots(nrows=len(ids_grid), ncols=len(ids_grid[0]), figsize=(16, 6), layout='tight')
for i, (indx, group) in enumerate(df_to_plot.groupby('Log ID')):
    if indx in ids_grid[0, 0]:
        ax = axs[0]
        ax.set_ylabel('Temperature Ratio $T_{S1} \;/\; T_{S2}$', fontsize=28)
    else:
        ax = axs[1]
        ax.set_xlim(-10, 200)
        
    timedelta = logs.loc[indx, 'timedelta']
    temp = stable_temperature(logs.loc[indx, 'alpha'], logs.loc[indx, 'phi'])
    label = f"$\Delta t = {timedelta}$"
    ax.plot(group['Steps'], group['T_ratio'], label=label)
    leg = ax.legend(ncol=1, fontsize=22, borderpad=1, title=f"$\\alpha = 0.25$\n\n$\phi={logs.loc[indx, 'phi']}$\n", title_fontsize=22)
    leg._legend_box.align = "left"
    # ax.set_xlim(0, 300)
    ax.set_xlabel('Interaction Step $k$', fontsize=28)
        
plt.show()

In [ ]:
fig.savefig(f"../img/obs_evolution/{STATE}/temperature_ratio_onTimes.eps")
fig.savefig(f"../img/obs_evolution/{STATE}/temperature_ratio_onTimes.png")

## Mean Photon Numbers

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(IMG_WIDTH/100 + 3, IMG_HEIGHT/100), layout='tight')
for i, (name, group) in enumerate(df.groupby('Log ID')):
    axs[i].plot(group['Time'], group['N1'], label='$S_1$')
    axs[i].plot(group['Time'], group['N2'], label='$S_2$')
    axs[i].set_xlabel('Time (a.u.)')
    axs[i].set_ylabel('Mean Photon Number')
    axs[i].set_title(f'Interaction Time = {name}')
    axs[i].legend()
plt.show()

## Entropies

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(IMG_WIDTH/100 + 3, IMG_HEIGHT/100), layout='tight')
ax_names = {
    0.1: r'$\Delta t = 0.1$',
    0.5: r'$\Delta t = 0.5$',
    1.0: r'$\Delta t = 1.0$',
}

y_names = {
    't01': r'$\mathcal{S}_V$',
    't02': r'$\mathcal{S}_{V1}$',
    't03': r'$\mathcal{S}_{V2}$'
}

for i, (name, group) in enumerate(df.groupby('Log ID')):
    for j, entropy in enumerate(['S', 'S1', 'S2']):
        axs[j].plot(group['Time'], group[entropy], 
                    #label=f'{ax_names[name]}'
                    )
        axs[j].set_xlabel('Time (a.u.)')
        axs[j].set_ylabel(f'{y_names[entropy]}')

axs[0].legend()

plt.show()
fig.savefig(f'../img/obs_evolution/{STATE}/entropy_evolution.eps', format='eps')

## Quantum Discord

In [ ]:
fig, ax = plt.subplots(figsize=(IMG_WIDTH/100, IMG_HEIGHT/100))

for name, group in df.groupby('Interaction Time'):
    ax.plot(group['Time'], group['Quantum Discord'], label=f'{ax_names[name]}')
ax.set_ylabel('$\mathcal{D}$')
ax.legend()

ax.set_xlim(0, 200)

plt.show()

fig.savefig(f'../img/obs_evolution/{STATE}/discord_evolution.eps', format='eps')

## Mutual Information

In [ ]:
fig, ax = plt.subplots(figsize=(IMG_WIDTH/100, IMG_HEIGHT/100))

for i, (name, group) in enumerate(df.groupby('Interaction Time')):
    ax.plot(group['Time'], group['Mutual Information'], label=ax_names[name])
ax.set_ylabel('$\mathcal{I}$')
ax.legend()

ax.set_xlim(0, 200)

plt.show()

fig.savefig(f'../img/obs_evolution/{STATE}/mutinf_evolution.eps', format='eps')

## One-way Classical Correlations

In [ ]:
fig, ax = plt.subplots(figsize=(IMG_WIDTH/100, IMG_HEIGHT/100))

for i, (name, group) in enumerate(df.groupby('Interaction Time')):
    ax.plot(group['Time'], group['Mutual Information'] - group['Quantum Discord'], label=ax_names[name])
ax.set_ylabel('$\mathcal{J}$')
ax.legend()

ax.set_xlim(0, 200)

plt.show()

fig.savefig(f'../img/obs_evolution/{STATE}/mutinf_evolution.eps', format='eps')

## Logarithmic Negativity

In [ ]:
fig, ax = plt.subplots(figsize=(IMG_WIDTH/100, IMG_HEIGHT/100))
for name, group in df.groupby('Interaction Time'):
    ax.plot(group['Time'], group['Logarithmic Negativity'], label=f'{ax_names[name]}')

## Heat Exchanges

In [ ]:
df_to_plot = df.loc[df['Time'] > 0]
fig, ax = plt.subplots(figsize=(3.5, 4.5))
for i, (name, group) in enumerate(df_to_plot.groupby('Interaction Time')):
    ax.plot(group['Time'], group['Jc'], label=ax_names[name])
ax.set_xlim([0, 100])
ax.set_ylim([-0.05, 0.00001])
ax.set_xlabel('Time (a.u.)')
ax.set_ylabel('$J_c$')
ax.legend(loc='center right', title='')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(3.5, 4.5))
for i, (name, group) in enumerate(df_to_plot.groupby('Interaction Time')):
    ax.plot(group['Time'], group['J1'], label=ax_names[name])
ax.set_xlim([0, 1000])
ax.set_ylim([-0.0002, 0.000005])
ax.set_xlabel('Time (a.u.)')
ax.set_ylabel('$J_1$')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(3.5, 4.5))
linestyles = ['-', '--', '-.', ':']
for i, (name, group) in enumerate(df_to_plot.groupby('Interaction Time')):
    ax.plot(group['Time'], group['J2'], label=name, linestyle=linestyles[i % len(linestyles)])
ax.set_xlim([0, 1000])
ax.set_ylim([-0.004, 0.0001])
ax.set_xlabel('Time (a.u.)')
ax.set_ylabel('$J_2$')

# Create an inset plot
axins = fig.add_axes([0.43, 0.25, 0.4, 0.45])
for i, (name, group) in enumerate(df_to_plot.groupby('Interaction Time')):
    axins.plot(group['Time'], group['J2'], label=name, linestyle=linestyles[i % len(linestyles)])
axins.set_xlim([0, 1000])
axins.set_ylim([-0.00005, 0.00001])

plt.show()

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(IMG_WIDTH/100 + 2, IMG_HEIGHT/100), layout='constrained')
ax_names = {
    0.1: '$\Delta t = 0.1$',
    0.5: '$\Delta t = 0.5$',
    1.0: '$\Delta t = 1.0$',
}

y_names = {
    'Jc': '$J_c$',
    'J1': '$J_1$',
    'J2': '$J_2$'
}

for i, (name, group) in enumerate(df.loc[df['Time'] > 0].groupby('Interaction Time')):
    for j, current in enumerate(['Jc', 'J1', 'J2']):
        axs[j].plot(group['Time'], group[current], label=ax_names[name])
        axs[j].set_xlabel('Time (a.u.)')
        axs[j].set_ylabel(y_names[current])
for ax in axs:
    ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
    ax.yaxis.get_offset_text().set_color('black')
        
axs[0].set_xlim([0, 1000])
axs[1].set_xlim([0, 1000])
axs[2].set_xlim([0, 200])
axs[0].set_ylim([-2e-4, 1e-5])
axs[1].set_ylim([-5e-5, 2e-6])
axs[2].set_ylim([-4e-3, 2e-4])
axs[0].legend()

# Create an inset plot
axins = fig.add_axes([0.9, 0.3, 0.15, 0.4])
for spine in axins.spines.values():
    spine.set_color('black')

for i, (name, group) in enumerate(df_to_plot.groupby('Interaction Time')):
    axins.plot(group['Time'], group['J2'], label=name)
axins.set_xlim([0, 1001])
axins.set_ylim([-0.00005, 0.00001])
axins.yaxis.get_offset_text().set_color('black')

plt.show()

fig.savefig(f'../img/obs_evolution/{STATE}/heat_evolution.eps', format='eps', bbox_inches='tight')

## Big Grid Plot

In [ ]:
df.head()

In [ ]:
ids_matrix = [
    [['T11', 'T16', 'T111'], ['T21', 'T26', 'T211']],
    [['K12', 'T17', 'T112'], ['T22', 'T27', 'T212']],
    [['T13', 'T18', 'T113'], ['T23', 'T28', 'T213']],
    [['T14', 'T19', 'T114'], ['T24', 'T29', 'T214']],
    [['T15', 'T110', 'T115'], ['T25', 'T210', 'T215']],
]
phi_dict = {
    '0': '\pi/2',
    '0.5235987755983': '\pi/6',
    '0.7853981633974': '\pi/4',
    '1.047197551197': '\pi/3',
    '1.570796326795': '\pi/2',
}
# Function to recover the stable temperature from the data
def stable_temperature(a, p):
    a = float(a)
    p = float(p)
    ga = 2*a**2
    gb = (1-a**2) * (1+np.cos(p))
    return - 1 / np.log(ga / gb)
# Make a grid of plots with 5 rows for phi and 2 columns for T
fig, axs = plt.subplots(len(ids_matrix), len(ids_matrix[0]), figsize=(12, 20), layout='constrained')

for i, row in enumerate(ids_matrix):
    for j, cell in enumerate(row):
        for k, log_id in enumerate(cell):
            alpha = logs.loc[log_id]['alpha']
            phi = logs.loc[log_id]['phi']
            temp = stable_temperature(alpha, phi)
            dt = logs.loc[log_id]['timedelta']
            data1 = df[df['Log ID'] == log_id]['N1']
            data2 = df[df['Log ID'] == log_id]['N2']
            time = df[df['Log ID'] == log_id]['Time']
            axs[i, j].plot(data1, label=f"$\Delta t$ = {dt}, $D_\\rightarrow$", color=STYLES['plotly'][k])
            axs[i, j].plot(data2, label=f"$\Delta t$ = {dt}, $D_\leftarrow$", color=STYLES['plotly'][k])
        axs[i, j].legend(prop={'size': 12})
        axs[i, j].set_title(f'$\\alpha = {alpha:.2f}$, $\phi = {phi_dict[phi]}$, $T = {temp:.2f}$')
        axs[i, j].set_xlabel('Time (a.u.)')
        axs[i, j].set_ylabel('$\mathcal{D}$')  #!!! Change with data
        axs[i, j].set_xlim([0, 5000])
        if j%2:
            # pass
            axs[i, j].set_ylim([2.6, 4.2])
        else:
            # pass
            axs[i, j].set_ylim([2.1, 3.1])

fig.suptitle('Quantum Discord Evolution\n', fontsize=20)  #!!! Change with data
plt.show()

## Random Interaction Times

In [ ]:
df['Sigma'] = df['Log ID'].apply(lambda x: x[2:])

def rmse(x):
    return np.sqrt(np.mean((x - np.mean(x))**2))

def iqr(x):
    """Interquartile range"""
    return np.percentile(x, 75) - np.percentile(x, 25)

def mad(x):
    """Median absolute deviation"""
    return np.median(np.abs(x - np.median(x)))

In [ ]:
fig, axs = plt.subplots(nrows=3, figsize=(IMG_WIDTH/100 + 3, 3*IMG_HEIGHT/100), layout='tight')

for i, (sigma, data) in enumerate(df.groupby('Sigma')):
    data['Bins'] = pd.cut(data['Time'], bins=200)
    data = data.groupby('Bins')['S'].agg(['mean', 'std', rmse])
    bin_edges = data.index.categories.left
    axs[i].plot(bin_edges, data['mean'], label=f'$\sigma = {sigma}$', ls='-')
    axs[i].fill_between(bin_edges, data['mean'] - data['rmse'], data['mean'] + data['rmse'], alpha=0.5)
    
plt.show()
    

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(IMG_WIDTH/100 + 3, IMG_HEIGHT/100), layout='tight')

for i, (sigma, data) in enumerate(df.groupby('Sigma')):
    ax_twin = axs[i].twinx()
    data['Bins'] = pd.cut(data['Time'], bins=200)
    
    data_qd = data.groupby('Bins')['Quantum Discord'].agg(['mean', 'std', mad, iqr, rmse])
    bin_edges = data_qd.index.categories.left
    
    axs[i].plot(bin_edges, data_qd['mean'], label=f'$\sigma = {sigma}$', ls='-')
    axs[i].fill_between(bin_edges, data_qd['mean'] - data_qd['mad'], data_qd['mean'] + data_qd['mad'], alpha=0.5)
    axs[i].set_xlim([0, 350])
    
    data_ratio = data.groupby('Bins')['T_ratio'].agg(['mean', 'std', mad, iqr, rmse])
    ax_twin.plot(bin_edges, data_ratio['mean'], label=f'$\sigma = {sigma}$', ls='-', color='red')
    ax_twin.fill_between(bin_edges, data_ratio['mean'] - data_ratio['mad'], data_ratio['mean'] + data_ratio['mad'], alpha=0.5, color='red')
    
plt.show()
    

In [ ]:
df.loc[df['Time'] < 1.050662]

In [ ]:
import qutip as qu

qu.tensor(qu.thermal_dm(3, 1), qu.thermal_dm(3, 1))

In [ ]:
qu.thermal_dm(20, 1)